In [1]:
import pandas as pd
import scipy.stats as stats
from functools import wraps

### Designing the AB test

In [2]:
def get_power(n, power_1, power_2, confidence_level):
    alpha = 1 - confidence_level
    qu = stats.norm.ppf(1 - alpha/2)
    diff = abs(power_2 - power_1)
    bp = (power_1 + power_2)/2
    v1 = power_1 * (1-power_1)
    v2 = power_2 * (1-power_2)
    bv = bp*(1-bp)
    
    temp_power_1 = stats.norm.cdf((n**0.5*diff - qu*(2*bv)**0.5) / (v1+v2)**0.5)
    temp_power_2 = 1 - stats.norm.cdf((n**0.5*diff + qu*(2*bv)**0.5) / (v1+v2)**0.5)
    
    power = temp_power_1 + temp_power_2
    
    return power

In [3]:
def get_sample_size(power, power_1, power_2, confidence_level, max_n):
    n=1
    while n <= max_n:
        
        temp_power = get_power(n, power_1, power_2, confidence_level)
        
        if temp_power >= power:
            return n
        else:
            n += 1

In [4]:
#test with p1 & p2 value
p1 = 0.1
p2 = 0.12
cl = 0.95

In [5]:
# Look at the impact of sample size increase on power
n_param_one = get_power(n=1000, power_1=p1, power_2=p2, confidence_level=cl)
n_param_two = get_power(n=2000, power_1=p1, power_2=p2, confidence_level=cl)

In [6]:
print(n_param_one)
print(n_param_two)

0.29808032538146
0.524515256115834


In [7]:
n1 = 1000
# Look at the impact of confidence level increase on power
alpha_param_one = get_power(n=n1, power_1=p1, power_2=p2, confidence_level=0.8)
alpha_param_two = get_power(n=n1, power_1=p1, power_2=p2, confidence_level=0.95)
    
# Compare the ratios
print(n_param_two / n_param_one)
print(alpha_param_one / alpha_param_two)

1.7596440001351992
1.8857367092232278


In [8]:
# Decrease of confidence level has more impact on the power than increasing the sample size 

### Defining functions for Hypothesis testing

In [9]:
def hypothesis_testing_evaluation(func):
    """returns p_value & hypothesis testing results"""
    
    @wraps(func)
    def wrapper_hypothesis(*args, **kwargs):
        p_value = func(*args, **kwargs)
        
        if p_value >= 0.05:
            print("Not significant test . Fail to reject the Null hypothesis")
        else:
            print("Snignificative test. May reject the Null hypothesis")
        
        return p_value
    
    return wrapper_hypothesis

In [10]:
@hypothesis_testing_evaluation
def get_p_value(control_conversion_rate, test_conversion_rate, control_size, test_size):
    lift = - abs(test_conversion_rate - control_conversion_rate)
    
    scale_control = control_conversion_rate * (1 - control_conversion_rate) * (1 / control_size)
    scale_test = test_conversion_rate * (1 - test_conversion_rate) * (1 / test_size)
    scale_val = (scale_control + scale_test)**0.5
    
    p_value = 2 * stats.norm.cdf(lift, loc=0, scale=scale_val)
    
    return p_value

In [11]:
# calculating p_value over 

In [12]:
p_value = get_p_value(control_conversion_rate=0.1, test_conversion_rate=0.17, control_size=1000, test_size=1000)
print(p_value)

Snignificative test. May reject the Null hypothesis
4.131297741047306e-06


In [13]:
# Get the p-value
p_value = get_p_value(control_conversion_rate=0.48, test_conversion_rate=0.5, control_size=1000, test_size=1000)
print(p_value)

Not significant test . Fail to reject the Null hypothesis
0.370901935824383


In [14]:
ab_test_results =pd.read_csv('../data/ab_test_results.csv')
# Compute and print the results
results = ab_test_results.groupby('group').agg({'uid':pd.Series.nunique}) 
print(results)

         uid
group       
GRP A  23009
GRP B  22874


In [15]:
def evaluate_null_hypo(func):
    'Return the conclusion to Null Hypothesis'

    @wraps(func)
    def wrapper_hypo(*args, **kwargs):

        p_value = func(*args, **kwargs)
        # Check for statistical significance
        if p_value >= 0.05:
            print("Not Significant. Fail to reject null hypothesis.")
        else:
            print("Significant Result. Reject null hypothesis.")
        return p_value

    return wrapper_hypo


@evaluate_null_hypo
def get_pvalue(con_conv, test_conv, con_size, test_size):  
    lift =  - abs(test_conv - con_conv)

    scale_one = con_conv * (1 - con_conv) * (1 / con_size)
    scale_two = test_conv * (1 - test_conv) * (1 / test_size)
    scale_val = (scale_one + scale_two)**0.5

    p_value = 2 * stats.norm.cdf(lift, loc = 0, scale = scale_val )

    return p_value


# Get the p-value
p_value = get_pvalue(con_conv=0.1, test_conv=0.17, con_size=1000, test_size=1000)
print(p_value)

# Get the p-value
p_value = get_pvalue(con_conv=0.48, test_conv=0.5, con_size=1000, test_size=1000)
print(p_value)

Significant Result. Reject null hypothesis.
4.131297741047306e-06
Not Significant. Fail to reject null hypothesis.
0.370901935824383


###### Conclusion part 1